In [6]:
# 1. Initiate libraries

import os
import requests
import torch

import numpy as np
# import open3d as o3d

import matplotlib.pyplot as plt

from transformers import AutoImageProcessor, AutoModelForDepthEstimation, pipeline
from PIL import Image

In [8]:
# 1.1. Read project modules

from depthanythingv1 import *
from openback import *
from watertight import *
from merge_meshes import *
from scale_meshes import *
from utils_colors import *
from utils_pointcloud import *

In [9]:
# 2. Initiate directories

# Directory with input images
rgb_dir = "rgb_dir"

# Directory to save .npy depth maps
depth_dir = "depth_dir"
os.makedirs(depth_dir, exist_ok=True)

# Directory to save colored depth maps
depth_colored_dir = "depth_colored_dir"
os.makedirs(depth_colored_dir, exist_ok=True)

# Directory to save black-and-white depth maps
depth_bw_dir = "depth_bw_dir"
os.makedirs(depth_bw_dir, exist_ok=True)

In [10]:
# 2.1. Choose colormap-scheme for colored depth map
colormap_scheme = "inferno"

In [12]:
# 3. Depth Estimation with DepthAnything V1

# Run depth estimation algorithm

batch_process_depth_estimation(rgb_dir=rgb_dir, depth_npy_dir=depth_dir, depth_bw_dir=depth_bw_dir, depth_colored_dir=depth_colored_dir, colormap_scheme=colormap_scheme)

OSError: cannot write mode F as PNG

In [4]:
# 3. Depth Estimation with DepthAnything V1

# 3.1. Initiatze checkpoints

checkpoints = [
    "Intel/zoedepth-nyu-kitti",
    "LiheYoung/depth-anything-large-hf",
    "jingheya/lotus-depth-g-v1-0",   
    "tencent/DepthCrafter"
    ]

checkpoint = "LiheYoung/depth-anything-large-hf"

# 3.2. Load models
image_processor = AutoImageProcessor.from_pretrained(checkpoint)
model = AutoModelForDepthEstimation.from_pretrained(checkpoint).to("cpu") 

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [5]:
# 3.3. Run depth estimation algorithm

for filename in sorted(os.listdir(rgb_dir)):
    if not filename.lower().endswith((".png", ".jpg", ".jpeg")):
        continue

    base = filename.rsplit(".", 1)[0]

    img_path = os.path.join(rgb_dir, filename)

    image = Image.open(img_path).convert("RGB")

    # Preprocesses
    inputs = image_processor(images=image, return_tensors="pt")

    # Predicts
    with torch.no_grad():
        outputs = model(**inputs)

    # Post-processes
    post_processed_output = image_processor.post_process_depth_estimation(
        outputs,
        target_sizes=[(image.height, image.width)]
    )

    predicted_depth = post_processed_output[0]["predicted_depth"]

    # Saves .npy
    npy_path = os.path.join(depth_dir, base + "_depth.npy")
    np.save(npy_path, predicted_depth.cpu().numpy())

    # Saves grayscale depth .png
    depth_norm = (predicted_depth - predicted_depth.min()) / (predicted_depth.max() - predicted_depth.min())
    depth = (depth_norm.cpu().numpy() * 255).astype("uint8")

    gray_path = os.path.join(depth_bw_dir, base + "_depth.png")

    Image.fromarray(depth).save(gray_path)

    # Saves colored depth .png

    colormap = plt.colormaps["inferno"]

    depth_colored = colormap(depth_norm.cpu().numpy())

    depth_colored = (depth_colored[:,:,:3] * 255).astype("uint8")

    colored_path = os.path.join(depth_colored_dir, base + "_depth_colored.png")

    Image.fromarray(depth_colored).save(colored_path)

    print(f"Saved {npy_path} and {gray_path}")

Saved depth_dir\sb1-obv_depth.npy and depth_bw_dir\sb1-obv_depth.png
Saved depth_dir\sb1-rev_depth.npy and depth_bw_dir\sb1-rev_depth.png
Saved depth_dir\sb2-obv_depth.npy and depth_bw_dir\sb2-obv_depth.png
Saved depth_dir\sb2-rev_depth.npy and depth_bw_dir\sb2-rev_depth.png


In [ ]:
# 4. 3D model creation

# 4.1. Non-watertight models
out_dir_openback = "3d_models_openback"
os.makedirs(out_dir_openback, exist_ok=True)

# Parameters
pixel_size = 1.0 # recommended: 1.0
relief_scale = 7.0 # recommended: 6.0 - 9.0
poisson_depth = 9 # recommended: 8-9
output_format = "obj" # recommended: "obj", "ply"

batch_process_openback(depth_dir, rgb_dir, out_dir_openback, output_format=output_format, pixel_size=pixel_size, relief_scale=relief_scale, poisson_depth=poisson_depth)

In [ ]:
# 4.2. Watertight models
out_dir_watertight = "3d_models_watertight"
os.makedirs(out_dir_watertight, exist_ok=True)

# Parameters
pixel_size = 1.0 # recommended: 1.0
relief_scale = 7.0 # recommended: 6.0 - 9.0
poisson_depth = 9 # recommended: 8-9
output_format = "obj" # recommended: "obj", "ply"

batch_process_watertight(depth_dir, rgb_dir, out_dir_watertight, output_format=output_format, pixel_size=pixel_size, relief_scale=relief_scale, poisson_depth=poisson_depth)

In [ ]:
# 4.3. Merged models
input_dir = "3d_models_openback"
output_dir_merged = "3d_models_merged"
os.makedirs(output_dir_merged, exist_ok=True)

# Parameters
gap_factor = 0.0 # recommended: 0.0
scale = True
scale_factor = 0.25

batch_process_merge(input_dir, output_dir, gap_factor=gap_factor, scale=scale, scale_factor=scale_factor)

In [ ]:
# 4.4. Scale models
input_dir_to_scale = "3d_models_merged"
output_dir_scaled = "3d_models_scaled"

# Parameters
scale_factor = 0.02

batch_scale_models(input_dir_to_scale, output_dir_scaled, scale_factor)